In [1]:
!python --version

Python 3.11.11


In [30]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from keras.models import load_model
from PIL import Image

%matplotlib inline

In [3]:
dataset_path = "dataset/others"

for filename in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, filename)
    try:
        img = Image.open(file_path)
        img_array = np.array(img)

        # Verifica se a imagem tem 2 canais
        if img_array.ndim == 3 and img_array.shape[2] == 2:
            print(f"Convertendo imagem de 2 canais para RGB: {filename}")
            img = img.convert("RGB")  # Converte para RGB
            img.save(file_path)  # Sobrescreve a imagem corrigida
        
    except Exception as e:
        print(f"Erro ao processar {filename}: {e}")

In [4]:
for filename in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, filename)
    try:
        img = Image.open(file_path)
        img_array = np.array(img)

        # Remove imagens que têm 2 canais
        if img_array.ndim == 3 and img_array.shape[2] == 2:
            print(f"Removendo imagem inválida: {filename}")
            os.remove(file_path)
        
    except Exception as e:
        print(f"Erro ao processar {filename}: {e}")

In [10]:
import os
import cv2
def remover_imagens_invalidas(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            img_path = os.path.join(root, file)
            try:
                img = cv2.imread(img_path)  # Carregar com OpenCV
                if img is None:
                    print(f"Removendo imagem corrompida: {img_path}")
                    os.remove(img_path)
                elif img.shape[-1] == 2:  # Se tiver 2 canais, é inválida
                    print(f"Removendo imagem com 2 canais: {img_path}")
                    os.remove(img_path)
            except Exception as e:
                print(f"Erro ao processar {img_path}: {e}")
                os.remove(img_path)

# Aplicar nos diretórios do dataset
remover_imagens_invalidas("dataset/others")
remover_imagens_invalidas("dataset/cat")


Removendo imagem corrompida: dataset/others\1308.jpg
Removendo imagem corrompida: dataset/others\1866.jpg
Removendo imagem corrompida: dataset/others\2384.jpg
Removendo imagem corrompida: dataset/cat\140.jpg


In [15]:
import os
from PIL import Image

def corrigir_imagens_invalidas(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            img_path = os.path.join(root, file)
            try:
                # Abrir imagem
                with Image.open(img_path) as img:
                    # Convertê-la para RGB
                    img = img.convert("RGB")
                    img.save(img_path)
            except Exception as e:
                print(f"Erro ao processar {img_path}: {e}")
                os.remove(img_path)

# Aplicar a correção no diretório
corrigir_imagens_invalidas("dataset/others")
corrigir_imagens_invalidas("dataset/cat")


Erro ao processar dataset/others\1259.jpg: [Errno 22] Invalid argument: 'dataset/others\\1259.jpg'


In [ ]:
# Caminho para o diretório com as imagens
dataset_path = "dataset"

# Criar conjuntos de treino e validação
train_dataset = image_dataset_from_directory(dataset_path,
                                             image_size=(224, 224),
                                             batch_size=32,
                                             validation_split=0.3,
                                             subset="training",
                                             seed=123)

val_dataset = image_dataset_from_directory(dataset_path,
                                           image_size=(224, 224),
                                           batch_size=32,
                                           validation_split=0.3,
                                           subset="validation",
                                           seed=123)

# Verificar o número de imagens carregadas
print(f"Número de imagens de treino: {len(train_dataset)}")
print(f"Número de imagens de validação: {len(val_dataset)}")

Found 4526 files belonging to 3 classes.
Using 3169 files for training.
Found 4526 files belonging to 3 classes.
Using 1357 files for validation.
Número de imagens de treino: 100
Número de imagens de validação: 43


In [17]:
# Carregar o modelo VGG16 sem a última camada
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Congelar as camadas do modelo base para evitar treinar tudo do zero
for layer in base_model.layers:
    layer.trainable = False

# Criar um novo modelo com camadas personalizadas
new_model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")  # 1 saída (gato ou não)
])

# Compilar o modelo
new_model.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

# Exibir o resumo da arquitetura
new_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,137,729 (80.63 MB)

 Trainable params: 6,423,041 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [18]:
new_model.fit(train_dataset, epochs=5, validation_data=val_dataset)

Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 290s 3s/step - accuracy: 0.8474 - loss: 9.2563 - val_accuracy: 0.9425 - val_loss: 1.0246
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 285s 3s/step - accuracy: 0.9484 - loss: 1.0409 - val_accuracy: 0.9359 - val_loss: 0.9051
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 299s 3s/step - accuracy: 0.9604 - loss: -0.5177 - val_accuracy: 0.9433 - val_loss: 1.3664
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 258s 3s/step - accuracy: 0.9561 - loss: -3.7059 - val_accuracy: 0.9440 - val_loss: 2.6852
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 243s 2s/step - accuracy: 0.9634 - loss: -5.1318 - val_accuracy: 0.9433 - val_loss: 3.6789


In [23]:
# Salvar o modelo treinado
new_model.save("vgg16_gatos.keras")  # Novo formato recomendado
model_loaded = load_model("vgg16_gatos.keras")

# Compilar novamente
model_loaded.compile(optimizer="adam",
                     loss="binary_crossentropy",
                     metrics=["accuracy"])


In [24]:
# Testar o modelo com novas imagens

# Caminho da pasta de teste
test_folder = "dataset/test"

# Formatos de imagem suportados
valid_formats = (".jpg", ".jpeg", ".png", ".bmp", ".gif")

# Listar todas as imagens na pasta
test_images = [f for f in os.listdir(test_folder) if f.lower().endswith(valid_formats)]

# Testar cada imagem da pasta
for img_name in test_images:
    img_path = os.path.join(test_folder, img_name)
    
    # Carregar e processar a imagem
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Adicionar dimensão batch
    img_array /= 255.0  # Normalizar

    # Fazer previsão
    prediction = model_loaded.predict(img_array)
    resultado = "É um gato!" if prediction[0] > 0.5 else "Não é um gato."

    # Exibir o resultado
    print(f"{img_name}: {resultado}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
test1.jpg: É um gato!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
test2.jpg: É um gato!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
test3.jpg: É um gato!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
test4.jpg: Não é um gato.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
test5.jpg: É um gato!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
test6.jpg: Não é um gato.
